In [81]:
import os
import zipfile
import pandas as pd
import torch
import torch.nn as nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
from torch.optim import Adam
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [2]:
# Path to your zip file
zip_path = os.path.expanduser('~/Downloads/archive(1).zip')
extract_dir = './flickr8k_data'

# Extract only if not already done
if not os.path.exists(extract_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

print("Extracted to:", extract_dir)
print("Contents:", os.listdir(extract_dir))

Extracted to: ./flickr8k_data
Contents: ['captions.txt', 'Images']


In [3]:
captions_path = os.path.join(extract_dir, 'captions.txt')
df = pd.read_csv(captions_path)

print(df.head())
print(f"\nNumber of unique images: {df['image'].nunique()}")

                       image  \
0  1000268201_693b08cb0e.jpg   
1  1000268201_693b08cb0e.jpg   
2  1000268201_693b08cb0e.jpg   
3  1000268201_693b08cb0e.jpg   
4  1000268201_693b08cb0e.jpg   

                                             caption  
0  A child in a pink dress is climbing up a set o...  
1              A girl going into a wooden building .  
2   A little girl climbing into a wooden playhouse .  
3  A little girl climbing the stairs to her playh...  
4  A little girl in a pink dress going into a woo...  

Number of unique images: 8091


In [4]:
# --- Image transform (standard ImageNet normalization) ---
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Dataset class for images ---
class FlickrImageDataset(Dataset):
    def __init__(self, image_dir, image_filenames, transform=None):
        self.image_dir = image_dir
        self.image_filenames = list(image_filenames)
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = self.image_filenames[idx]
        path = os.path.join(self.image_dir, img_name)
        # wrap in try-except to catch corrupt images
        try:
            image = Image.open(path).convert("RGB")
        except Exception as e:
            # if image fails to open, create a black image instead and log
            print(f"Failed to open {path}: {e}")
            image = Image.new('RGB', (224,224))
        if self.transform:
            image = self.transform(image)
        return image, img_name

In [5]:
# --- Model setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*(list(resnet.children())[:-1]))  # remove FC layer
resnet.to(device)
resnet.eval()

# --- Create dataset and dataloader ---
image_dir = os.path.join(extract_dir, "Images")
unique_images = df["image"].unique()
image_dataset = FlickrImageDataset(image_dir, unique_images, transform)
image_loader = DataLoader(image_dataset, batch_size=32, shuffle=False, num_workers=2)

C:\Users\12089\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\12089\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# test model forward
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*(list(resnet.children())[:-1])).to(device).eval()

C:\Users\12089\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\12089\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
image_filenames = sorted(os.listdir(image_dir))   # better to use df['image'].unique() if you want same order
dataset = FlickrImageDataset(image_dir, image_filenames, transform)
loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)  # num_workers=0 is safest

image_features_list = []
image_names = []

In [15]:
with torch.no_grad():
    for imgs, names in tqdm(loader, desc="Extracting image features"):
        imgs = imgs.to(device)
        feats = resnet(imgs)               # (B, 2048, 1, 1)
        feats = feats.view(feats.size(0), -1)  # (B, 2048)
        image_features_list.append(feats.cpu())
        image_names.extend(names)

image_features = torch.cat(image_features_list, dim=0).numpy()
print("Done. image_features shape:", image_features.shape)

Extracting image features: 100%|██████████| 253/253 [03:10<00:00,  1.33it/s]

Done. image_features shape: (8091, 2048)


In [17]:
from transformers import BertTokenizer, BertModel

# --- Device and model setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')
bert.to(device)
bert.eval()

# --- Dataset for captions ---
class FlickrCaptionDataset(Dataset):
    def __init__(self, captions):
        self.captions = captions
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        text = self.captions[idx]
        # Return plain text; we’ll tokenize in collate_fn for batching
        return text

# --- Custom collate_fn to batch tokenize ---
def collate_fn(batch_texts):
    return tokenizer(batch_texts, return_tensors='pt',
                     truncation=True, padding=True, max_length=64)


In [19]:
# --- Create dataset and dataloader ---
caption_dataset = FlickrCaptionDataset(df["caption"].tolist())
caption_loader = DataLoader(
    caption_dataset,
    batch_size=32,         
    shuffle=False,
    num_workers=0,         
    collate_fn=collate_fn  
)

In [21]:
# --- Extract features ---
caption_features = []

with torch.no_grad():
    for batch in tqdm(caption_loader, desc="Extracting caption features"):
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = bert(**inputs)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]  # (B, 768)
        caption_features.append(cls_embeddings.cpu())

caption_features = torch.cat(caption_features, dim=0).numpy()

print("Caption features shape:", caption_features.shape)

Extracting caption features: 100%|██████████| 1265/1265 [05:01<00:00,  4.19it/s]

Caption features shape: (40455, 768)


In [23]:
# Map image filename to its index
image_to_idx = {name: i for i, name in enumerate(image_names)}

# For each caption row, find which image it corresponds to
caption_to_image_idx = df["image"].map(image_to_idx).values

print("caption_to_image_idx shape:", caption_to_image_idx.shape)
print("Example mapping:", list(zip(df['caption'][:3], caption_to_image_idx[:3])))

caption_to_image_idx shape: (40455,)
Example mapping: [('A child in a pink dress is climbing up a set of stairs in an entry way .', 0), ('A girl going into a wooden building .', 0), ('A little girl climbing into a wooden playhouse .', 0)]


In [29]:
np.save("flickr8k_image_features.npy", image_features)
np.save("flickr8k_caption_features.npy", caption_features)
np.save("flickr8k_caption_to_image.npy", caption_to_image_idx)
np.save("flickr8k_image_names.npy", np.array(image_names))

In [33]:
# Split on images
n_images = len(image_names)
indices = np.arange(n_images)

In [39]:
# 70/15/15 split
train_idx, temp_idx = train_test_split(indices, test_size=0.30, random_state=42, shuffle=True)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42, shuffle=True)
print(f"Train images: {len(train_idx)}, Val: {len(val_idx)}, Test: {len(test_idx)}")

Train images: 5663, Val: 1214, Test: 1214


In [41]:
# Create masks for captions 
caption_to_image_idx = caption_to_image_idx.astype(int)

train_mask = np.isin(caption_to_image_idx, train_idx)
val_mask = np.isin(caption_to_image_idx, val_idx)
test_mask = np.isin(caption_to_image_idx, test_idx)

# Split image features 
image_train = image_features[train_idx]
image_val   = image_features[val_idx]
image_test  = image_features[test_idx]

# Split captions (and keep their alignment)
caption_train = caption_features[train_mask]
caption_val   = caption_features[val_mask]
caption_test  = caption_features[test_mask]

In [43]:
# Link captions to local image indices within each split
def remap_caption_indices(global_indices, split_indices):
    """
    Convert global image indices in caption_to_image_idx to 0..len(split_indices)-1 within that split.
    """
    mapping = {g: i for i, g in enumerate(split_indices)}
    return np.array([mapping[i] for i in global_indices if i in mapping])

In [45]:
caption_to_train_img = remap_caption_indices(caption_to_image_idx[train_mask], train_idx)
caption_to_val_img   = remap_caption_indices(caption_to_image_idx[val_mask], val_idx)
caption_to_test_img  = remap_caption_indices(caption_to_image_idx[test_mask], test_idx)

In [47]:
# Sanity check
print("Train split shapes:")
print("  Image features:", image_train.shape)
print("  Caption features:", caption_train.shape)
print("  Caption→Image indices:", caption_to_train_img.shape)

Train split shapes:
  Image features: (5663, 2048)
  Caption features: (28315, 768)
  Caption→Image indices: (28315,)


In [49]:
np.save("train_image_features.npy", image_train)
np.save("val_image_features.npy", image_val)
np.save("test_image_features.npy", image_test)

np.save("train_caption_features.npy", caption_train)
np.save("val_caption_features.npy", caption_val)
np.save("test_caption_features.npy", caption_test)

np.save("train_caption_to_image.npy", caption_to_train_img)
np.save("val_caption_to_image.npy", caption_to_val_img)
np.save("test_caption_to_image.npy", caption_to_test_img)

In [57]:
# Now time to set up and train the actual multi-modal autoencoder
config = {
    "image_feat_path": "train_image_features.npy",   # we'll load proper files below
    "caption_feat_path": "train_caption_features.npy",
    "caption_to_image_path": "train_caption_to_image.npy",
    "val_image_feat_path": "val_image_features.npy",
    "val_caption_feat_path": "val_caption_features.npy",
    "val_caption_to_image_path": "val_caption_to_image.npy",
    "latent_dim": 512,
    "img_input_dim": 2048,
    "txt_input_dim": 768,
    "img_hidden": 1024,
    "txt_hidden": 512,
    "batch_size": 128,    # try 128; lower if memory limited (e.g., 64)
    "lr": 1e-3,
    "weight_decay": 1e-5,
    "epochs": 40,
    "lambda_align": 1.0,  # weight for latent alignment loss; tuneable
    "checkpoint_dir": "./corr_ae_checkpoints",
    "seed": 42,
}

In [59]:
torch.manual_seed(config["seed"])
np.random.seed(config["seed"])

os.makedirs(config["checkpoint_dir"], exist_ok=True)

In [61]:
#I don't need to reload these if we run it all in the same notebook but I'm pasting the load functions here anyway
image_train = np.load(config["image_feat_path"])
caption_train = np.load(config["caption_feat_path"])
cap2img_train = np.load(config["caption_to_image_path"])

image_val = np.load(config["val_image_feat_path"])
caption_val = np.load(config["val_caption_feat_path"])
cap2img_val = np.load(config["val_caption_to_image_path"])

print("Shapes (train):", image_train.shape, caption_train.shape, cap2img_train.shape)
print("Shapes (val):", image_val.shape, caption_val.shape, cap2img_val.shape)

Shapes (train): (5663, 2048) (28315, 768) (28315,)
Shapes (val): (1214, 2048) (6070, 768) (6070,)


In [63]:
# 3) Compute train-set normalization (mean/std) and apply to all splits
# Normalize per-feature (column-wise) using training set statistics
img_mean = image_train.mean(axis=0, keepdims=True)
img_std = image_train.std(axis=0, keepdims=True) + 1e-6

txt_mean = caption_train.mean(axis=0, keepdims=True)
txt_std = caption_train.std(axis=0, keepdims=True) + 1e-6

In [65]:
def normalize_images(x):
    return (x - img_mean) / img_std

def normalize_texts(x):
    return (x - txt_mean) / txt_std

image_train = normalize_images(image_train)
image_val   = normalize_images(image_val)

caption_train = normalize_texts(caption_train)
caption_val   = normalize_texts(caption_val)

In [67]:
# 4) Dataset that returns paired (image_feat, caption_feat) for each caption
class CaptionImagePairedDataset(Dataset):
    """
    Iterates over captions. For index i, returns:
      caption_features[i], image_features[ caption_to_image_idx[i] ]
    """
    def __init__(self, caption_feats, image_feats, caption_to_image_idx):
        assert len(caption_feats) == len(caption_to_image_idx)
        self.caption_feats = caption_feats.astype(np.float32)
        self.image_feats = image_feats.astype(np.float32)
        self.cap2img = caption_to_image_idx.astype(np.int64)

    def __len__(self):
        return len(self.caption_feats)

    def __getitem__(self, idx):
        cap = self.caption_feats[idx]
        img = self.image_feats[self.cap2img[idx]]
        return {"image": torch.from_numpy(img), "caption": torch.from_numpy(cap)}

train_dataset = CaptionImagePairedDataset(caption_train, image_train, cap2img_train)
val_dataset   = CaptionImagePairedDataset(caption_val, image_val, cap2img_val)

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=0)

In [73]:
# 5) Model: two autoencoders with shared latent dimension
class ImageAE(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=1024, latent_dim=512):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        recon = self.decoder(z)
        return z, recon

class TextAE(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=512, latent_dim=512):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        recon = self.decoder(z)
        return z, recon

In [75]:
# Instantiate models
img_ae = ImageAE(
    input_dim=config["img_input_dim"],
    hidden_dim=config["img_hidden"],
    latent_dim=config["latent_dim"]
).to(device)

txt_ae = TextAE(
    input_dim=config["txt_input_dim"],
    hidden_dim=config["txt_hidden"],
    latent_dim=config["latent_dim"]
).to(device)

# 6) Losses and optimizer
recon_loss_fn = nn.MSELoss()    # reconstruction for both
align_loss_fn = nn.MSELoss()    # align latents

params = list(img_ae.parameters()) + list(txt_ae.parameters())
optimizer = Adam(params, lr=config["lr"], weight_decay=config["weight_decay"])

In [77]:
# 7) Training / validation loop
def run_epoch(loader, training=True):
    if training:
        img_ae.train(); txt_ae.train()
    else:
        img_ae.eval(); txt_ae.eval()

    total_recon_img = 0.0
    total_recon_txt = 0.0
    total_align = 0.0
    total_loss = 0.0
    n_samples = 0

    pbar = tqdm(loader, desc="train" if training else "val")
    with torch.set_grad_enabled(training):
        for batch in pbar:
            imgs = batch["image"].to(device)    # shape (B, img_dim)
            caps = batch["caption"].to(device)  # shape (B, txt_dim)
            batch_size = imgs.shape[0]

            # forward
            z_img, img_recon = img_ae(imgs)
            z_txt, txt_recon = txt_ae(caps)

            # losses
            L_img = recon_loss_fn(img_recon, imgs)
            L_txt = recon_loss_fn(txt_recon, caps)
            L_align = align_loss_fn(z_img, z_txt)

            loss = L_img + L_txt + config["lambda_align"] * L_align

            if training:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            total_recon_img += L_img.item() * batch_size
            total_recon_txt += L_txt.item() * batch_size
            total_align += L_align.item() * batch_size
            total_loss += loss.item() * batch_size
            n_samples += batch_size

            pbar.set_postfix({
                "loss": f"{total_loss / n_samples:.4f}",
                "Limg": f"{total_recon_img / n_samples:.4f}",
                "Ltxt": f"{total_recon_txt / n_samples:.4f}",
                "Lalign": f"{total_align / n_samples:.4f}"
            })

    return {
        "loss": total_loss / n_samples,
        "Limg": total_recon_img / n_samples,
        "Ltxt": total_recon_txt / n_samples,
        "Lalign": total_align / n_samples
    }

In [79]:
best_val_loss = float("inf")

for epoch in range(1, config["epochs"] + 1):
    print(f"\n=== Epoch {epoch}/{config['epochs']} ===")
    train_metrics = run_epoch(train_loader, training=True)
    val_metrics = run_epoch(val_loader, training=False)

    print(f"Train loss: {train_metrics['loss']:.4f} | Val loss: {val_metrics['loss']:.4f}")

    # Save checkpoint (every epoch)
    ckpt = {
        "epoch": epoch,
        "img_state": img_ae.state_dict(),
        "txt_state": txt_ae.state_dict(),
        "optimizer": optimizer.state_dict(),
        "train_metrics": train_metrics,
        "val_metrics": val_metrics,
        "config": config
    }
    ckpt_path = os.path.join(config["checkpoint_dir"], f"corr_ae_epoch{epoch}.pt")
    torch.save(ckpt, ckpt_path)

    # Keep best
    if val_metrics["loss"] < best_val_loss:
        best_val_loss = val_metrics["loss"]
        torch.save(ckpt, os.path.join(config["checkpoint_dir"], "corr_ae_best.pt"))
        print("Saved best checkpoint.")

print("Training finished.")


=== Epoch 1/40 ===


val: 100%|██████████| 48/48 [00:00<00:00, 116.35it/s, loss=0.6464, Limg=0.3226, Ltxt=0.2340, Lalign=0.0897]


Train loss: 0.9268 | Val loss: 0.6464
Saved best checkpoint.

=== Epoch 2/40 ===


val: 100%|██████████| 48/48 [00:00<00:00, 114.00it/s, loss=0.5140, Limg=0.2508, Ltxt=0.1895, Lalign=0.0738]


Train loss: 0.5338 | Val loss: 0.5140
Saved best checkpoint.

=== Epoch 3/40 ===


...

Train loss: 0.1920 | Val loss: 0.2672

=== Epoch 39/40 ===


val: 100%|██████████| 48/48 [00:00<00:00, 123.06it/s, loss=0.2606, Limg=0.1656, Ltxt=0.0562, Lalign=0.0389]


Train loss: 0.1927 | Val loss: 0.2606
Saved best checkpoint.

=== Epoch 40/40 ===


val: 100%|██████████| 48/48 [00:00<00:00, 119.64it/s, loss=0.2623, Limg=0.1653, Ltxt=0.0581, Lalign=0.0388]


Train loss: 0.1897 | Val loss: 0.2623
Training finished.


In [83]:
#Now time to evaluate on the validation set
#Probably don't need to reload the model, but I'm going to include the code again in case we break this up into more managable files
# --- Load best checkpoint ---
best_ckpt_path = os.path.join(config["checkpoint_dir"], "corr_ae_best.pt")
ckpt = torch.load(best_ckpt_path, map_location=device)

img_ae.load_state_dict(ckpt["img_state"])
txt_ae.load_state_dict(ckpt["txt_state"])
img_ae.eval(); txt_ae.eval()

print(f"Loaded best checkpoint from {best_ckpt_path} (epoch {ckpt['epoch']})")

Loaded best checkpoint from ./corr_ae_checkpoints\corr_ae_best.pt (epoch 39)


In [85]:
# Encode into latent space
with torch.no_grad():
    # Encode images
    Z_imgs = []
    for i in range(0, image_val.shape[0], 256):
        batch = torch.from_numpy(image_val[i:i+256]).float().to(device)
        z, _ = img_ae(batch)
        Z_imgs.append(z.cpu().numpy())
    Z_imgs = np.concatenate(Z_imgs, axis=0)   # shape (N_images, latent_dim)

    # Encode captions
    Z_caps = []
    for i in range(0, caption_val.shape[0], 256):
        batch = torch.from_numpy(caption_val[i:i+256]).float().to(device)
        z, _ = txt_ae(batch)
        Z_caps.append(z.cpu().numpy())
    Z_caps = np.concatenate(Z_caps, axis=0)   # shape (N_captions, latent_dim)

print("Encoded latent shapes:", Z_imgs.shape, Z_caps.shape)

Encoded latent shapes: (1214, 512) (6070, 512)


In [87]:
#Use Recall@1/5/10 to evaluate hyperparameter performance
#Note that we are using cosine similarity
#Should we consider using L2 metric instead? Does this even make sense?
def retrieval_metrics(Z_caps, Z_imgs, caption_to_image_idx):
    sims = cosine_similarity(Z_caps, Z_imgs)  # (num_caps, num_imgs)
    ranks = []
    for i, true_img_idx in enumerate(caption_to_image_idx):
        sim_scores = sims[i]
        sorted_indices = np.argsort(-sim_scores)  # descending
        rank = np.where(sorted_indices == true_img_idx)[0][0] + 1
        ranks.append(rank)

    ranks = np.array(ranks)
    recall_at_1  = np.mean(ranks <= 1)
    recall_at_5  = np.mean(ranks <= 5)
    recall_at_10 = np.mean(ranks <= 10)
    med_rank = np.median(ranks)

    return {
        "Recall@1": recall_at_1,
        "Recall@5": recall_at_5,
        "Recall@10": recall_at_10,
        "MedianRank": med_rank
    }

metrics_val = retrieval_metrics(Z_caps, Z_imgs, cap2img_val)
for k, v in metrics_val.items():
    print(f"{k}: {v:.4f}")

Recall@1: 0.0524
Recall@5: 0.1633
Recall@10: 0.2540
MedianRank: 38.0000


In [93]:
#Quick visualization of what images are retrieved by what caption:
image_dir = "path/to/Flickr8k/Images"  # e.g. "/content/Flickr8k/Images"

def show_top_images_for_caption(caption_idx, top_k=5):
    """
    Show top-k retrieved validation images for a given caption index.
    """
    # Get the embedding for this caption
    caption_embedding = Z_caps[caption_idx].reshape(1, -1)
    sims = cosine_similarity(caption_embedding, Z_imgs)[0]
    top_img_indices = np.argsort(-sims)[:top_k]

    # Print caption text
    print(f"\nCAPTION: {caption_val[caption_idx]}")
    true_img_idx = cap2img_val[caption_idx]
    print(f"TRUE IMAGE: {image_val[true_img_idx]} (index {true_img_idx})")
    print(f"Top {top_k} retrieved images:")

    # Display images
    plt.figure(figsize=(15, 4))
    for i, img_idx in enumerate(top_img_indices):
        img_name = image_val[img_idx]  # this assumes image_val contains filenames
        img_path = os.path.join(image_dir, img_name)
        try:
            img = Image.open(img_path)
        except Exception as e:
            print(f"Could not open {img_path}: {e}")
            continue
        plt.subplot(1, top_k, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Rank {i+1}")
    plt.show()

In [95]:
for i in random.sample(range(len(caption_val)), 3):
    show_top_images_for_caption(i, top_k=5)


CAPTION: [-4.40105468e-01  2.62257129e-01 -1.82232273e+00  1.71197605e+00
  1.61076617e+00  1.87989962e+00 -1.62736058e+00  1.18145227e+00
 -2.05603933e+00  1.02855766e+00  1.01329672e+00 -7.16462910e-01
 -9.71074820e-01  8.97090316e-01 -3.40079486e-01  6.10017776e-01
 -9.27111983e-01 -1.01340437e+00 -4.90179360e-01 -2.48719901e-01
 -4.23995793e-01  1.04763293e+00 -1.71742356e+00  6.69775307e-01
 -4.73519862e-01  1.22269928e+00  6.58022404e-01 -8.49948704e-01
  1.39369681e-01  1.06526160e+00  3.10615599e-01 -1.74752653e+00
  7.66426086e-01  6.06213868e-01 -3.13661933e-01 -1.16403329e+00
 -1.45711675e-01  2.63895333e-01 -1.61355817e+00 -9.26913738e-01
  1.97226977e+00  3.24242890e-01 -5.30107796e-01 -1.21021128e+00
  1.24355778e-01 -5.41140616e-01  5.43014467e-01  9.64143932e-01
  1.34140587e+00  8.16520154e-01 -7.91564763e-01 -5.19880056e-01
  5.68725288e-01  3.08898352e-02  6.28872156e-01  1.61347345e-01
  1.35990477e+00 -6.41580582e-01  2.30055881e+00  4.66104656e-01
 -3.53251338e-0

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'ndarray'

<Figure size 1500x400 with 0 Axes>